# Model architecture

In this notebook, we will build our model and have it ready to train. We will create 3 types of models:
        
#### (1) Session Specific:

These models will try to learn from one session's worth of data. The intuition why we decided to attempt was because of the fact that there is significant variability within sessions for the same subjects.

#### (2) Subject Specific:
These models will be trained by pooling all of the sessions' data into a big dataset. However keeps subjects apart as there is also significant between-subject variability.

#### (3) General:
This model will pool all of the data into one giant dataset and learn from every subjects every session.

#### (4) Hybrid:
This model will pool session specific models with averages.

In [1]:
from packages import *
%matplotlib inline

/home/kaandonbekci/src/modules/utils.py:2: DeprecationWarning: Python2 support is deprecated and will be removed in a future release. Consider switching to Python3.
  import os, nilearn.plotting, nibabel
/home/kaandonbekci/.local/lib/python2.7/site-packages/nilearn/plotting/js_plotting_utils.py:14: ImportWarning: Not importing directory 'html': missing __init__.py
  from html import escape  # Unavailable in Py2
/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binar

In [2]:
# from get_activations import get_activations
import random

In [3]:
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model
K.set_floatx('float32')
from tensorflow.keras.applications import vgg16
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Conv3D, MaxPooling3D, AveragePooling3D, Flatten, Input, Lambda, concatenate, Dropout, Conv3DTranspose
from tensorflow.keras.models import load_model, save_model
from tensorflow.keras.regularizers import l1, l2

In [4]:
run = True
save = True
load = True
check = True

Y_is_pixel = False
output_dim = 7500

In [5]:
subject = 1
version = 'n-drop'

In [6]:
# vgg= vgg16.VGG16(weights='imagenet', include_top=True)

In [7]:
# vgg.summary()

In [8]:
# layers = vgg.layers
# for layer in layers:
#     print layer.name
#     print layer.__dict__
#     print 
#     print

## (1) Session models


In [9]:
def init_bold_session_model(subject):
          
    # CHANNELS LAST
    channels = 1
    batch_size = 1
    
    voxel_input = Input(shape = (12168, ), name='voxel_input')
        
    bold_input = Input(shape=(52, 43, 45, channels), name='bold_input')

    x = Conv3D(filters=32, kernel_size=3, name='conv_1_1', padding='same', activation='relu')(bold_input)

    x = Conv3D(filters=32, kernel_size=3, name='conv_1_2', padding='same', activation='relu')(x)
    
    x = Conv3D(filters=32, kernel_size=3, name='conv_1_3', padding='same', activation='relu')(x)
    
    x = Conv3D(filters=32, kernel_size=3, name='conv_1_4', padding='same', activation='relu')(x)


    x = MaxPooling3D(pool_size= 2, name='pool_1')(x)
#     x = AveragePooling3D(pool_size= 2, name='pool_1')(x)
#     x_p_1_ = AveragePooling3D(pool_size= 2, name='pool_4')(x_p_1)

    x = Conv3D(filters=64, kernel_size=5, name='conv_2_1', padding='same', activation='relu')(x)
    
    x = Conv3D(filters=64, kernel_size=5, name='conv_2_2', padding='same', activation='relu')(x)
    
#     x = Conv3D(filters=64, kernel_size=5, name='conv_2_3', padding='same', activation='relu')(x)

#     x = Conv3D(filters=32, kernel_size=3, name='conv_2_4', padding='same', activation='relu')(x)
    
#     x = Conv3D(filters=32, kernel_size=3, name='conv_2_5', padding='same', activation='relu')(x)
    
#     x = Conv3D(filters=32, kernel_size=3, name='conv_2_6', padding='same', activation='relu')(x)
    
#     x = Conv3D(filters=32, kernel_size=3, name='conv_2_7', padding='same', activation='relu')(x)

    x = MaxPooling3D(pool_size=2, name='pool_2')(x)
    
#     x = Conv3DTranspose(filters= 64, kernel_size=5, strides=2, name='deconv_1_1', padding='same', activation='relu')(x)
    
#     x = Conv3DTranspose(filters= 64, kernel_size=5, name='deconv_1_2', padding='same', activation='relu')(x)

#     x = Conv3DTranspose(filters= 64, kernel_size=5, name='deconv_1_3', padding='same', activation='relu')(x)
    
#     x = Conv3DTranspose(filters= 64, kernel_size=3, strides=2, name='deconv_2_1', padding='same', activation='relu')(x)
    
#     x = Conv3DTranspose(filters= 64, kernel_size=3, name='deconv_2_2', padding='same', activation='relu')(x)
    
    x = Conv3D(filters=128, kernel_size=3, name='conv_3_1', padding='same', activation='relu')(x)
  
    x = Conv3D(filters=128, kernel_size=3, name='conv_3_2', padding='same', activation='relu')(x)
    
    x = MaxPooling3D(pool_size=2, name='pool_3')(x)
    
#     x = Flatten(x)
    
#     x = Conv3D(filters=128, kernel_size=3, name='conv_5_1', padding='same', activation='relu')(x)
    
#     x = Conv3D(filters=128, kernel_size=3, name='conv_5_2', padding='same', activation='relu')(x)
    
#     x = Conv3D(filters=64, kernel_size=3, name='conv_3_3', padding='same', activation='relu')(x)
    
#     x = MaxPooling3D(pool_size=2, name='pool_3')(x)
#     x = AveragePooling3D(pool_size=2, name='pool_4')(x)
    
#     x = 

    
    x = Flatten(name='flatten_1')(x)
    
    x = concatenate(inputs = [x, voxel_input])
        
#     x = Dense(units=output_dim ,name='dense_1', activation='relu')(x) #, kernel_regularizer=l1(.01))(x)
    
#     x = Dense(units=7500 ,name='dense_2', activation='relu', kernel_regularizer=l1(.01))(x)
    
    x = Dropout(.15, name='drop_2')(x)
    
    out = Dense(output_dim, activation='relu', name = 'predictions', kernel_regularizer=l1(.01))(x) #, kernel_regularizer=l1(.01))(x)
    bold_convnet = Model(inputs = [bold_input, voxel_input], outputs = out)
    
    return bold_convnet

In [10]:
sub1_conv = init_bold_session_model(1)

In [11]:
sub1_conv.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bold_input (InputLayer)         (None, 52, 43, 45, 1 0                                            
__________________________________________________________________________________________________
conv_1_1 (Conv3D)               (None, 52, 43, 45, 3 896         bold_input[0][0]                 
__________________________________________________________________________________________________
conv_1_2 (Conv3D)               (None, 52, 43, 45, 3 27680       conv_1_1[0][0]                   
__________________________________________________________________________________________________
conv_1_3 (Conv3D)               (None, 52, 43, 45, 3 27680       conv_1_2[0][0]                   
__________________________________________________________________________________________________
conv_1_4 (

In [12]:
plot_model(sub1_conv)

In [13]:
if save:
    sub1_conv.save(os.path.join(UNTRAINED_MODELS_DIR, 'sub-{}_mod-{}'.format(subject,version)))
    print 'Saved model.'

Saved model.


In [29]:
25*18*21*64

321300

In [20]:
12*9*10*64

69120

In [22]:
6*4*5*128

15360